# Gemma-7b 활용/ 요약 모델 만들기

In [2]:
#import os
# # 서버의 GPU 1번 사용하게 만들기 
#os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # 1번 GPU만 노출됨
import torch

# 원하는 GPU를 지정 (예: 두 번째 GPU 사용)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Using device: cuda:0


## 1. 데이터셋 구축

In [5]:
import os
import re
import glob
import zipfile
from datasets import load_dataset

def load_and_build_dataset_from_zip(zip_path, extracted_dir):
    """
    zip 파일 내의 JSON 파일들을 동적으로 검색하여 데이터셋을 구성합니다.
    
    1. 지정된 경로에 압축 파일을 해제합니다.
    2. glob 모듈을 사용해 모든 JSON 파일을 검색합니다.
    3. 정규표현식을 활용하여 파일명에 'train' (대소문자 구분 없이)이 포함된 파일만 필터링합니다.
    4. 필터링된 파일들을 이용해 Hugging Face의 load_dataset 함수로 데이터셋을 구성합니다.
    
    :param zip_path: 압축 파일의 경로
    :param extracted_dir: 압축 해제할 디렉토리
    :return: 구성된 데이터셋
    """
    # 압축 해제할 디렉토리가 없으면 생성합니다.
    if not os.path.exists(extracted_dir):
        os.makedirs(extracted_dir)
    
    # zip 파일을 열어 지정된 디렉토리에 압축 해제
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extracted_dir)
    
    # glob 모듈을 사용하여 extracted_dir 내의 모든 JSON 파일 검색
    json_files = glob.glob(os.path.join(extracted_dir, "*.json"))
    
    # 정규표현식을 사용하여 파일명에 'train'이 포함된 파일만 필터링 (대소문자 무시)
    train_files = [f for f in json_files if re.search(r"train", os.path.basename(f), re.IGNORECASE)]
    
    if not train_files:
        raise ValueError("압축 해제된 디렉토리에서 'train'이 포함된 JSON 파일을 찾을 수 없습니다.")
    
    # validation 파일도 필요하면 비슷한 방식으로 필터링 가능 (예시)
    validation_files = [f for f in json_files if re.search(r"validation", os.path.basename(f), re.IGNORECASE)]
    
    # 데이터 파일 딕셔너리 생성: load_dataset은 리스트로 전달받은 여러 파일들을 자동으로 병합합니다.
    data_files = {"train": train_files}
    if validation_files:
        data_files["validation"] = validation_files
    
    # JSON 파일들로부터 데이터셋 구성
    dataset = load_dataset("json", data_files=data_files)
    return dataset

# 사용 예시:
zip_path = "/home/wanted-1/potenup-workspace/Project/project3/team2/학습데이터/06_문서요약 텍스트/Training/법률_train_original.zip"
extracted_dir = "/home/wanted-1/potenup-workspace/Project/project3/team2/학습데이터/06_문서요약 텍스트/Training/extracted"

dataset = load_and_build_dataset_from_zip(zip_path, extracted_dir)
print(dataset)


Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['name', 'delivery_date', 'documents'],
        num_rows: 24329
    })
})


In [6]:
def extract_original_and_summary(document):
    """
    주어진 document 딕셔너리에서 원문과 요약문을 추출합니다.
    
    원문:
        - 'text' 필드는 리스트 안에 리스트 형태로 구성되어 있습니다.
        - 각 내부 딕셔너리의 'sentence' 값을 추출하여 하나의 문자열로 결합합니다.
    요약문:
        - 'abstractive' 필드에 요약문이 문자열 리스트로 제공되며,
          첫 번째 요소를 최종 요약문으로 사용합니다.
    """
    # 원문 추출: 'text' 필드의 각 문장을 결합합니다.
    original_sentences = []
    for block in document.get("text", []):
        for sentence_info in block:
            sentence = sentence_info.get("sentence", "")
            if sentence:
                original_sentences.append(sentence.strip())
    original_text = " ".join(original_sentences)
    
    # 요약문 추출: 'abstractive' 필드의 첫 번째 요소 사용
    summary_list = document.get("abstractive", [])
    summary_text = summary_list[0].strip() if summary_list else ""
    
    return original_text, summary_text

# 전체 데이터셋의 모든 문서에서 원문과 요약문을 추출하는 반복문
# dataset['train']['documents'] 가 모든 문서를 포함하는 리스트라고 가정합니다.
all_originals = []
all_summaries = []

for document in dataset['train']['documents']:
    original_text, summary_text = extract_original_and_summary(document)
    all_originals.append(original_text)
    all_summaries.append(summary_text)

# # 추출된 결과를 문서별로 출력합니다.
# for idx, (orig, summ) in enumerate(zip(all_originals, all_summaries)):
#     print(f"Document {idx+1}")
#     print("Original Text:")
#     print(orig)
#     print("Summary:")
#     print(summ)
#     print("-" * 40)

In [7]:
import pandas as pd

# 예를 들어, 전체 문서에서 추출한 원문과 요약문 리스트가 있다고 가정합니다.
data = {
    "original_text": all_originals,
    "summary_text": all_summaries
}
df = pd.DataFrame(data)

# DataFrame 내용 확인
# print(df.head())

# CSV 파일로 저장하기
df.to_csv("extracted_documents.csv", index=False, encoding="utf-8-sig")


## 2. 데이터셋 포맷팅

In [8]:
import pandas as pd
from datasets import Dataset

# CSV 파일 불러오기 (UTF-8 인코딩 사용, 필요시 'utf-8-sig' 또는 'euc-kr'로 변경)
csv_path = "extracted_documents.csv"
df = pd.read_csv(csv_path, encoding="utf-8-sig")

# DataFrame 컬럼 이름 변경: 원문과 요약 컬럼명을 일관성 있게 변경합니다.
df = df.rename(columns={"original_text": "input_text", "summary_text": "target_text"})

# 데이터 전처리 예시: 한국어 텍스트 정제를 위한 간단한 함수 추가
def clean_korean_text(text):
    # 불필요한 공백 제거, 특수문자 처리 등 추가적인 처리가 필요하면 여기에 구현
    return text.strip()

# 전처리 적용: 각 원문과 요약문에 대해 정제 함수를 적용
df["input_text"] = df["input_text"].apply(clean_korean_text)
df["target_text"] = df["target_text"].apply(clean_korean_text)

# Pandas DataFrame을 Hugging Face Dataset으로 변환합니다.
dataset = Dataset.from_pandas(df)
print(dataset[0])


{'input_text': '원고가 소속회사의 노동조합에서 분규가 발생하자 노조활동을 구실로 정상적인 근무를 해태하고, 노조조합장이 사임한 경우, 노동조합규약에 동 조합장의 직무를 대행할 자를 규정해 두고 있음에도 원고 자신이 주동하여 노조자치수습대책위원회를 구성하여 그 위원장으로 피선되어 근무시간중에도 노조활동을 벌여 운수업체인 소속회사의 업무에 지장을 초래하고 종업원들에게도 나쁜 영향을 끼쳐 소속회사가 취업규칙을 위반하고 고의로 회사업무능률을 저해하였으며 회사업무상의 지휘명령에 위반하였음을 이유로 원고를 징계해고 하였다면, 이는 원고의 노동조합 활동과는 관계없이 회사취업규칙에 의하여 사내질서를 유지하기 위한 사용자 고유의 징계권에 기하여 이루어진 정당한 징계권의 행사로 보아야 한다.', 'target_text': '원고가  주동하여 회사업무능률을 저해하고 회사업무상의 지휘명령에 위반하였다면 이에 따른 징계해고는 사내질서를 유지하기 위한 사용자 고유의 정당한 징계권의 행사로 보아야 한다.'}


In [9]:
# 4. 전처리 함수 정의 (batched=True)
def preprocess_function(examples):
    """
    각 예제에 대해 프롬프트와 타겟 텍스트를 생성합니다.
    - 'input_text' 컬럼에 저장된 원문 앞에 "### 원문:"과 "### 요약:" 구분자를 추가합니다.
    - 'target_text' 컬럼은 그대로 요약문으로 사용합니다.
    """
    prompts = []
    targets = []
    for document, summary in zip(examples["input_text"], examples["target_text"]):
        prompt = "### 원문:\n" + document + "\n\n### 요약:"
        prompts.append(prompt)
        targets.append(summary)
    return {"input_text": prompts, "target_text": targets}

# 5. 데이터셋에 전처리 함수 적용
tokenized_dataset = dataset.map(preprocess_function, batched=True)
print("\nTokenized Dataset Sample:")
print(tokenized_dataset[0])

Map:   0%|          | 0/24329 [00:00<?, ? examples/s]


Tokenized Dataset Sample:
{'input_text': '### 원문:\n원고가 소속회사의 노동조합에서 분규가 발생하자 노조활동을 구실로 정상적인 근무를 해태하고, 노조조합장이 사임한 경우, 노동조합규약에 동 조합장의 직무를 대행할 자를 규정해 두고 있음에도 원고 자신이 주동하여 노조자치수습대책위원회를 구성하여 그 위원장으로 피선되어 근무시간중에도 노조활동을 벌여 운수업체인 소속회사의 업무에 지장을 초래하고 종업원들에게도 나쁜 영향을 끼쳐 소속회사가 취업규칙을 위반하고 고의로 회사업무능률을 저해하였으며 회사업무상의 지휘명령에 위반하였음을 이유로 원고를 징계해고 하였다면, 이는 원고의 노동조합 활동과는 관계없이 회사취업규칙에 의하여 사내질서를 유지하기 위한 사용자 고유의 징계권에 기하여 이루어진 정당한 징계권의 행사로 보아야 한다.\n\n### 요약:', 'target_text': '원고가  주동하여 회사업무능률을 저해하고 회사업무상의 지휘명령에 위반하였다면 이에 따른 징계해고는 사내질서를 유지하기 위한 사용자 고유의 정당한 징계권의 행사로 보아야 한다.'}


### 데이터 정보 확인

In [10]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# CSV 파일 불러오기 (파일 경로를 실제 경로로 수정하세요)
csv_path = "extracted_documents.csv"
df = pd.read_csv(csv_path, encoding="utf-8")

# 데이터프레임 구조와 컬럼 정보 출력
print("데이터 샘플:")
print(df.head())
print("\n데이터 정보:")
print(df.info())

# 숫자형 컬럼 추출
numeric_cols = df.select_dtypes(include=["int64", "float64"]).columns

# 숫자형 컬럼이 존재하는지 확인
if len(numeric_cols) == 0:
    print("\n숫자형 컬럼이 없습니다. 해당 시각화를 건너뜁니다.")
else:
    # 1. 수치형 컬럼 분포: 히스토그램을 통해 각 수치형 컬럼의 분포를 파악
    df[numeric_cols].hist(figsize=(12, 10), bins=20)
    plt.suptitle("수치형 컬럼 히스토그램")
    plt.show()

    # 2. 이상치 탐지를 위한 박스플롯: 각 수치형 컬럼의 분포와 이상치를 확인
    plt.figure(figsize=(15, 5))
    for i, col in enumerate(numeric_cols, 1):
        plt.subplot(1, len(numeric_cols), i)
        sns.boxplot(x=df[col])
        plt.title(f"{col} 박스플롯")
    plt.tight_layout()
    plt.show()

    # 3. 밀도 플롯: 수치형 컬럼의 분포를 부드러운 곡선으로 시각화
    plt.figure(figsize=(12, 6))
    for col in numeric_cols:
        sns.kdeplot(df[col].dropna(), shade=True, label=col)
    plt.title("수치형 컬럼 밀도 플롯")
    plt.xlabel("값")
    plt.ylabel("밀도")
    plt.legend()
    plt.show()

데이터 샘플:
                                       original_text  \
0  원고가 소속회사의 노동조합에서 분규가 발생하자 노조활동을 구실로 정상적인 근무를 해...   
1  수출입업체인 원고가 의류제품을 제조ㆍ수출함에 있어 같은 그룹내 종합무역상사인 소외 ...   
2  가등기담보권자가 제소전 화해조항에 따라 자기 명의로 소유권이전의 본등기를 경료한 후...   
3  가. 부가가치세법 제22조 제3항 단서에 제1호와 제2호가 동시에 해당한다는 뜻은 ...   
4  소득세법 제116조 제1항의 규정에 의하면 정부는 과세표준확정신고를 하여야 할 자에...   

                                        summary_text  
0  원고가  주동하여 회사업무능률을 저해하고 회사업무상의 지휘명령에 위반하였다면 이에 ...  
1  수출입업체인 원고가 의류제품을 제조ㆍ수출함에 있어 소외 회사의 직수출실적을 지원하기...  
2  가등기담보권자가 제소전 화해조항에 의해 자기 명의로 소유권이전의 본등기를 경료하고 ...  
3  부가가치세법 제22조 제3항 단서에 제1호와 제2호가 동시에 해당한다는 의미는 제1...  
4  소득세법 제116조 제1항에 따르면 정부는 과세표준확정신고를 해야 할 자에 대해 당...  

데이터 정보:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24329 entries, 0 to 24328
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   original_text  24329 non-null  object
 1   summary_text   24329 non-null  object
dtypes: object(2)
memory usage: 380.

## ⭐3. 모델 및 토크나이저 가져오기

In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import get_peft_model, LoraConfig, TaskType

def load_model_and_tokenizer(model_name="google/gemma-7b-it"):
    """
    4비트 양자화와 PEFT(LoRA)를 적용하여 모델과 토크나이저를 로드합니다.
    4비트 양자화 설정은 BitsAndBytesConfig에서 load_in_4bit=True를 사용하며,
    추가 설정으로 연산 dtype, 이중 양자화 사용 여부, 양자화 타입을 지정합니다.
    """
    # 4-bit 양자화 설정
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.float16,  # 연산 시 사용할 데이터 타입
        bnb_4bit_use_double_quant=True,         # 이중 양자화 사용 (메모리 효율 개선)
        bnb_4bit_quant_type="nf4"               # 양자화 타입: "nf4" 또는 "fp4" 중 선택 (일반적으로 "nf4" 추천)
    )
    # device_map = "auto"
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    # 토크나이저와 양자화된 모델 로드
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config=bnb_config,
        #device_map=device_map
    )
    
    # 4-bit 양자화된 모델은 기본적으로 파인튜닝이 불가능하므로, LoRA 어댑터를 추가합니다.
    lora_config = LoraConfig(
        r=8,
        lora_alpha=32,
        target_modules=["q_proj", "v_proj"],  # 모델 구조에 맞게 조정 (예: Gemma 모델의 경우)
        lora_dropout=0.1,
        bias="none",
        task_type=TaskType.CAUSAL_LM,
    )
    model = get_peft_model(model, lora_config)
    
    return model, tokenizer

# 사용 예시:
model, tokenizer = load_model_and_tokenizer("google/gemma-7b-it")
print("Model and tokenizer loaded with 4-bit quantization and LoRA applied.")



/home/wanted-1/miniconda3/envs/whisper/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
`low_cpu_mem_usage` was None, now default to True since model is quantized.
Loading checkpoint shards: 100%|██████████| 4/4 [00:05<00:00,  1.30s/it]


Model and tokenizer loaded with 4-bit quantization and LoRA applied.


## ⭐4. 모델 튜닝하기

In [4]:
import os
import torch
from transformers import (
    AutoTokenizer, 
    AutoModelForCausalLM, 
    BitsAndBytesConfig, 
    TrainingArguments, 
    Trainer, 
    DataCollatorForSeq2Seq
)
from datasets import load_dataset

### 1) 모델 및 토크나이저 로드

In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

def load_model_and_tokenizer(model_name="google/gemma-7b-it"):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name)
    return model, tokenizer


# def load_model_and_tokenizer(model_name="google/gemma-7b-it"):
#     """
#     BitsAndBytesConfig를 사용하여 8-bit quantization과 CPU offload를 적용해 모델을 로드합니다.
#     """
#     bnb_config = BitsAndBytesConfig(
#         load_in_8bit=True,           
#         llm_int8_threshold=6.0,
#         llm_int8_has_fp16_weight=False,
#         llm_int8_enable_fp32_cpu_offload=True  # CPU offload 활성화
#     )
    
#     # device_map은 "auto"를 사용하여 내부적으로 offload 전략이 적용되도록 함
#     device_map = "auto"
    
#     tokenizer = AutoTokenizer.from_pretrained(model_name)
#     model = AutoModelForCausalLM.from_pretrained(
#         model_name,
#         quantization_config=bnb_config,
#         device_map=device_map
#     )
#     return model, tokenizer


### ⭐2) 데이터셋 준비

In [6]:
def prepare_dataset(csv_path, encoding="utf-8"):
    """
    CSV 파일을 불러와서 데이터셋을 구축합니다.
    CSV 파일에는 'original_text'와 'summary_text' 컬럼이 있다고 가정하며,
    이를 'input_text'와 'target_text'로 재정의합니다.
    """
    import pandas as pd
    from datasets import Dataset
    
    df = pd.read_csv(csv_path, encoding=encoding)
    df = df.rename(columns={"original_text": "input_text", "summary_text": "target_text"})
    dataset = Dataset.from_pandas(df)
    return dataset

def tokenize_dataset(dataset, tokenizer, max_input_length=1024, max_target_length=256):
    """
    데이터셋의 각 예제에 대해 입력과 타겟 텍스트를 토큰화합니다.
    """
    def tokenize_function(example):
        model_inputs = tokenizer(example["input_text"], max_length=max_input_length, truncation=True)
        with tokenizer.as_target_tokenizer():
            labels = tokenizer(example["target_text"], max_length=max_target_length, truncation=True)
        model_inputs["labels"] = labels["input_ids"]
        return model_inputs
    
    tokenized_dataset = dataset.map(tokenize_function, batched=True, remove_columns=dataset.column_names)
    return tokenized_dataset

### ⭐3) 학습 설정 및 실행

In [7]:
def train_model(model, tokenizer, tokenized_dataset, output_dir="./results", epochs=3, seed=42):
    """
    토큰화된 데이터셋을 학습용과 평가용으로 분할하고, Trainer를 사용하여 모델을 fine-tuning합니다.
    """
    # 데이터셋 분할
    split_dataset = tokenized_dataset.train_test_split(test_size=0.1, seed=seed)
    train_dataset = split_dataset["train"]
    eval_dataset = split_dataset["test"]
    
    # Data Collator 설정
    data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)
    
    # TrainingArguments 설정
    training_args = TrainingArguments(
        output_dir=output_dir,
        evaluation_strategy="epoch",
        learning_rate=5e-5,
        num_train_epochs=epochs,
        weight_decay=0.01,
        fp16=True,
        logging_steps=50,
        save_total_limit=3,
        seed=seed
    )
    
    # Trainer 객체 생성
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        tokenizer=tokenizer,
        data_collator=data_collator
    )
    
    # 학습 실행
    trainer.train()
    
    # 학습 완료 후 모델 저장
    trainer.save_model(os.path.join(output_dir, "gemma_finetuned_final"))
    
    return trainer

### ⭐4) 메인 함수: 전체 파이프라인 실행

In [ ]:
def main():
    # 0. GPU 설정 (필요한 경우 환경 변수나 코드 내 device 설정)
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("Using device:", device)
    
    # 1. 모델 및 토크나이저 로드 (모델은 이미 device_map="auto"로 올바른 디바이스에 로드됨)
    model, tokenizer = load_model_and_tokenizer("google/gemma-7b-it")
    # model.to(device)  # 8-bit 모델에서는 이 호출이 필요하지 않으며, 에러가 발생합니다.
    
    # 2. 데이터셋 준비 및 토큰화
    csv_path = "extracted_documents.csv"  # CSV 파일 경로 지정
    raw_dataset = prepare_dataset(csv_path, encoding="utf-8")
    tokenized_dataset = tokenize_dataset(raw_dataset, tokenizer)
    
    # 3. 모델 튜닝
    train_model(model, tokenizer, tokenized_dataset, output_dir="./results", epochs=3, seed=42)

if __name__ == "__main__":
    main()


Using device: cuda:0


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]/home/wanted-1/miniconda3/envs/whisper/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3970: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Map: 100%|██████████| 1000/1000 [00:00<00:00, 9289.42 examples/s]
/home/wanted-1/miniconda3/envs/whisper/lib/python3.10/site-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/tmp/ipykernel_2181768/3916027822.py:27: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


OutOfMemoryError: CUDA out of memory. Tried to allocate 288.00 MiB. GPU 0 has a total capacity of 23.63 GiB of which 265.00 MiB is free. Including non-PyTorch memory, this process has 23.24 GiB memory in use. Of the allocated memory 22.78 GiB is allocated by PyTorch, and 26.34 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

: 

In [1]:
def main():
    # 0. GPU 설정 (필요한 경우 환경 변수나 코드 내 device 설정)
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("Using device:", device)
    
    # 1. 모델 및 토크나이저 로드 (모델은 이미 device_map="auto"로 올바른 디바이스에 로드됨)
    model, tokenizer = load_model_and_tokenizer("google/gemma-7b-it")
    # model.to(device)  # 8-bit 모델에서는 이 호출이 필요하지 않으며, 에러가 발생합니다.
    
    # 2. 데이터셋 준비 및 토큰화
    csv_path = "extracted_documents.csv"  # CSV 파일 경로 지정
    raw_dataset = prepare_dataset(csv_path, encoding="utf-8")
    tokenized_dataset = tokenize_dataset(raw_dataset, tokenizer)
    
    # 3. 모델 튜닝
    train_model(model, tokenizer, tokenized_dataset, output_dir="./results", epochs=3, seed=42)

if __name__ == "__main__":
    main()


NameError: name 'torch' is not defined

## 5. 프롬프트 및 하이퍼파라미터 설정하기

In [ ]:
# def generate_summary(input_text, model, tokenizer, max_new_tokens=150):
#     """
#     Fine-tuning된 모델을 이용해 프롬프트에 따른 요약을 생성하는 함수입니다.
#     생성 파라미터(temperature, top_p 등)는 실험 결과에 따라 조정할 수 있습니다.
#     """
#     inputs = tokenizer(input_text, return_tensors="pt").to(model.device)
#     with torch.no_grad():
#         outputs = model.generate(
#             input_ids=inputs["input_ids"],
#             attention_mask=inputs["attention_mask"],
#             max_new_tokens=max_new_tokens,
#             do_sample=True,              # 무작위성 유지 (필요에 따라 결정적 생성도 고려)
#             temperature=0.7,
#             top_p=0.8,
#             repetition_penalty=1.2,
#             no_repeat_ngram_size=3,
#             pad_token_id=tokenizer.pad_token_id or tokenizer.eos_token_id
#         )
#     return tokenizer.decode(outputs[0], skip_special_tokens=True)

# # 프롬프트 예시 (경제/금융 주제)
# sample_input = (
#     "### 원문:\n"
#     "최근 한국 경제는 글로벌 경기 둔화와 내수 부진 등으로 어려움을 겪고 있으며, "
#     "정부와 기업들은 다양한 정책과 혁신을 통해 위기를 극복하고자 노력하고 있습니다.\n\n"
#     "### 요약:"
# )
# sample_summary = generate_summary(sample_input, model, tokenizer, max_new_tokens=150)
# print("Generated Summary:\n", sample_summary)


NameError: name 'model' is not defined

## 6. 출력 결과 확인 후 프롬프트 및 하이퍼 파라미터 조정하기

In [ ]:
# def evaluate_and_adjust():
#     """
#     생성된 결과를 평가하고, 프롬프트 문구나 생성 파라미터를 조정하는 반복 과정을 수행합니다.
#     이 단계는 수동 또는 자동화된 평가 루프를 통해 진행할 수 있습니다.
#     """
#     # 예시: 생성 결과 평가 (실제 평가 기준은 ROUGE, BLEU 등 메트릭을 활용할 수 있음)
#     print("출력 결과를 확인하고, 프롬프트 및 파라미터를 조정하세요.")
#     # 필요 시, 재생성 및 파라미터 변경 로직을 추가합니다.

# # 평가 및 조정 호출 예시
# evaluate_and_adjust()


## 7. 튜닝 루프 만들기

In [ ]:
# def tuning_loop(model, tokenizer, trainer, iterations=3):
#     """
#     4~6단계를 반복하는 튜닝 루프입니다.
#     iterations 값은 실험에 따라 조정합니다.
#     각 반복에서 모델의 출력 결과를 평가하고,
#     필요 시 프롬프트 문구 및 하이퍼파라미터를 조정하여 최적의 결과를 도출합니다.
#     """
#     for i in range(iterations):
#         print(f"\n=== Tuning iteration {i+1} ===")
#         # 프롬프트 및 하이퍼파라미터 설정
#         sample_input = (
#             "### 원문:\n"
#             "최근 한국 경제는 글로벌 경기 둔화와 내수 부진 등으로 어려움을 겪고 있으며, "
#             "정부와 기업들은 다양한 정책과 혁신을 통해 위기를 극복하고자 노력하고 있습니다.\n\n"
#             "### 요약:"
#         )
#         output = generate_summary(sample_input, model, tokenizer, max_new_tokens=150)
#         print("Generated Output:\n", output)
        
#         # 평가 및 필요 시 파라미터, 프롬프트 조정 (여기서는 단순히 출력 확인)
#         evaluate_and_adjust()
    
#     print("최종 튜닝 완료.")

# # 튜닝 루프 실행 예시
# tuning_loop(model, tokenizer, trainer, iterations=3)
